### RDD 의 2가지 operation, spark 가 지연된 연산을 사용하는 이유
### Transformation and Action

## 1. Transformation
결과값을 새로운 RDD로 return
resource ↑ (Lazy execution) 지연 실행

## 2. Action
결과값을 출력 or 저장
resource ↓ (Eager execution) 즉시 실행

### 두 개의 방식으로 나뉜 이유? -> 지연된 연산이 유용한 경우가 있기 때문.

1 - 메모리를 최대한 활용하려고.

연산을 지연시켜서 디스크,네트워크의 연산을 최소화한다.

예를 들어 딥러닝과 머신러닝을 하는 경우에 데이터로 모델을 학습시키는데,

이 때 데이터를 다루는 task 의 빈도수가 늘어나게된다. (ex : batch size)

지연된 연산을 하지 않을 경우에 work-flow 는 

task -> disk -> task -> disk .... 처럼 반복을 해야만 하는 비효율적인 상황이 일어난다. 

아니면 메모리에 저장시켜놓을 데이터를 미리 지정해두어야 한다.

transformation 의 경우 지연된 연산을 하기 때문에 모든 데이터를 메모리에 미리 로드할 이유가 없고, 

실제로 그 행위가 필요한 시점까지 미루어 최적의 계획, 데이터 work-flow 를 최적화 한다.

그리고 저장시킬 데이터들은 driver -> worker nodes 의 흐름으로 데이터를 적절히 파티셔닝 후 분산 저장, 병렬 처리하게 되고

최종적으로 저장된 데이터를 사용할 때엔 합쳐진 하나의 것처럼 사용 가능하다.

요약하면 최적의 계획으로 데이터를 저장하고 필요하지 않은 네트워크 통신, resource 를 줄인다고 볼 수 있다.

In [1]:
# 데이터를 메모리에 저장하고 사용하는 메소드 

# .cache(), .persist()

from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName('restaurant-average')
sc = SparkContext(conf = conf)

In [17]:
directory = '/Users/admin/Desktop/spark_hadoop/data-engineering/01-spark'
filename = 'restaurant_reviews.csv'

lines = sc.textFile(f"file:///{directory}/{filename}")

header = lines.first()

filtered_lines = lines.filter(lambda x:x != header)

In [18]:
def parse(row):
    col = row.split(',')
    category = col[2]
    reviews = int(col[3])
    return (category, reviews)

category_review = filtered_lines.map(parse)

result1 = category_review.take(10)
result2 = category_review.mapValues(lambda x:(x, 1)).collect()

위의 category_review 라는 RDD는 2 번의 연산이 진행된 것인데, 

.persist() 메소드를 쓴다면 1 번의 연산만 해도 되는 장점이 있는 것이다.

In [13]:
category_review = filtered_lines.map(parse).persist()

### 머신러닝의 경우 지연 연산이 도움

Regression 의 경우 gradient-descent 를 할 때의 손실함수는 Least Square Method(LSE) 최소제곱법과 lr 로 오차를 점점 줄이게 되는데,

이 때 데이터를 계속해서 참조하게 되므로 지연 연산을 할 수 있도록 .cache() 메소드를 걸어주는 것이 좋겠다.

spark 를 이용해서 머신러닝 파이프라인을 만든다고 가정했을 때 이러면 좋지 않을까?

In [ ]:
# 예시 코드 
hdfs_file_path = "hdfs://<namenode_h>:<namenode_p>/path/to/hdfs_file"

hdfsRDD = sc.textFile(hdfs_file_path).map(parse).cache() # RDD

위와 같이 한다면 불필요한 네트워크 통신을 최소화하고 resource 도 절약할 수 있을 것이다.

또한 '메모리에 어디까지 저장할 것인가' 에 대한 level 까지 지정할 수 있는 storage level 도 구글링 해보면 좋을 것 같다.

cache 의 경우 default storage level, RDD : Memory_only, dataframe : memory_and_disk 이고,
persis 의 경우 storage level 은 사용자가 커스텀한다.